# Exercise_Multi_Device_Plugin_and_the_DevCloud

In this exercise, you will load a model on to three devices: NCS, GPU and CPU and perform an inference on it. We will then calculate the time it takes to do the same.



#### Set up paths so we can run Dev Cloud utilities
You *must* run this every time they enter a Workspace session.

In [1]:
%env PATH=/opt/conda/bin:/opt/spark-2.4.3-bin-hadoop2.7/bin:/opt/conda/bin:/usr/local/sbin:/usr/local/bin:/usr/sbin:/usr/bin:/sbin:/bin:/opt/intel_devcloud_support
import os
import sys
sys.path.insert(0, os.path.abspath('/opt/intel_devcloud_support'))
sys.path.insert(0, os.path.abspath('/opt/intel'))

env: PATH=/opt/conda/bin:/opt/spark-2.4.3-bin-hadoop2.7/bin:/opt/conda/bin:/usr/local/sbin:/usr/local/bin:/usr/sbin:/usr/bin:/sbin:/bin:/opt/intel_devcloud_support


## The model

We will be using the `vehicle-license-plate-detection-barrier-0106` model for this exercise. Remember that to run a model on the GPU, we need to use FP16 as the model precision.

The model is present in the `/data/models/intel` folder.

# Step 1: Creating a Python Script

The first step is to create a python script that you can use to load the model and perform an inference. I have used the `writefile` magic to create a python file called `inference.py`. You will need to complete this file.

In [27]:
%%writefile inference.py

import time
import numpy as np
import cv2
from openvino.inference_engine import IENetwork
from openvino.inference_engine import IECore
from openvino.inference_engine import IEPlugin
import argparse

def main(args):
    model=args.model_path
    model_weights=model+'.bin'
    model_structure=model+'.xml'
    
    start=time.time()
    
    
    # TODO: Load the model
    print("Starting...")
    print("Device:", args.device)
    print("Model:", args.model_path)

    core = IECore()
    print("Core created")
    
    network = IENetwork(model_structure, model_weights)
    print("Network created")
    
    exec_network = core.load_network(network, args.device, num_requests=1)
    print("Network loaded")

    print(f"Time taken to load model = {time.time()-start} seconds")
    
    # Reading and Preprocessing Image
    input_img=cv2.imread('car.png')
    input_img=cv2.resize(input_img, (300,300), interpolation = cv2.INTER_AREA)
    input_img=np.moveaxis(input_img, -1, 0)

    
    # TODO: Prepare the model for inference (create input dict etc.)
    input_key = next(iter(network.inputs))
    input_dict = {input_key:input_img}
    print("Input created")

    
    start=time.time()
    for _ in range(10):
        # TODO: Run Inference in a Loop
        exec_network.infer(input_dict)
        print(f"{i} Inference completed")
        
    
    print(f"Time Taken to run 10 Infernce on GPU is = {time.time()-start} seconds")

if __name__=='__main__':
    parser=argparse.ArgumentParser()
    parser.add_argument('--model_path', required=True)
    parser.add_argument('--device', default=None)
    
    args=parser.parse_args() 
    main(args)

Overwriting inference.py


## Step 2: Creating a job submission script

To submit a job to the devcloud, we need to create a script. I have named the script as `inference_gpu_model_job.sh`.

Can you write a script that will take the model path and device as a command line argument and then call the python file you created in the previous cell with the path to the model?

In [28]:
%%writefile inference_job.sh

#TODO: Create job submission script
exec 1>/output/stdout.log 2>/output/stderr.log
mkdir -p /output

DEVICE=$1
MODELPATH=$2

# Run the load model python script\n",
python3 inference.py  --model_path ${MODELPATH} --device ${DEVICE}

cd /output
tar zcvf output.tgz stdout.log stderr.log

Overwriting inference_job.sh


## Step 3: Running on the Device

In the cell below, can you write the qsub command that will submit your job to the Device?

In [29]:
DEVICE_CPU = 'CPU'
DEVICE_GPU = 'GPU'
DEVICE_VPU = 'MYRIAD'
DEVICE_MULTI = 'MULTI:MYRIAD,GPU,CPU'
MODEL_FP32 = '/data/models/intel/vehicle-license-plate-detection-barrier-0106/FP32/vehicle-license-plate-detection-barrier-0106'
MODEL_FP16 = '/data/models/intel/vehicle-license-plate-detection-barrier-0106/FP16/vehicle-license-plate-detection-barrier-0106'
NODE_CPU = 'tank-870:e3-1268l-v5'
NODE_GPU = 's002-n011:i5-6500te:intel-hd-530'
NODE_VPU = 'tank-870:i5-6500te:intel-ncs2'
NODE_MULTI = 'tank-870:i5-6500te:intel-hd-530:intel-ncs2'

#job_id = !qsub inference_job.sh -d . -l nodes=1:{NODE_CPU} -F "{DEVICE_CPU} {MODEL_FP32}" -N store_core
#job_id = !qsub inference_job.sh -d . -l nodes=1:{NODE_GPU} -F "{DEVICE_GPU} {MODEL_FP16}" -N store_core
job_id = !qsub inference_job.sh -d . -l nodes=1:{NODE_MULTI} -F "{DEVICE_MULTI} {MODEL_FP16}" -N store_core

print(job_id[0])


5tK72tgqSTUShVSG4JkVPfAYA1kvRhPE


## Step 4: Getting the Live Stat Values

By running the below command, we can see the live status of the commands.

In [30]:
import liveQStat
liveQStat.liveQStat()

## Step 5: Get the results

Running the cell below will get the output files from our job

In [31]:
import get_results

get_results.getResults(job_id[0], filename="output.tgz", blocking=True)
# get_results.getResults(job_id_core[0], get_stderr=True, blocking=True)
# get_results.getResults(job_id_core[0], blocking=True)

getResults() is blocking until results of the job (id:5tK72tgqSTUShVSG4JkVPfAYA1kvRhPE) are ready.
Please wait...................................Success!
output.tgz was downloaded in the same folder as this notebook.


In [24]:
!tar zxf output.tgz

In [25]:
!cat stdout.log

Starting...
Device: MULTI:MYRIAD,GPU,CPU
Model: /data/models/intel/vehicle-license-plate-detection-barrier-0106/FP16/vehicle-license-plate-detection-barrier-0106
Plugin created
Network created


In [26]:
!cat stderr.log

Traceback (most recent call last):
  File "inference.py", line 73, in <module>
    main(args)
  File "inference.py", line 39, in main
    exec_network = plugin.load(network=network, num_requests=1)
  File "ie_api.pyx", line 1205, in openvino.inference_engine.ie_api.IEPlugin.load
  File "ie_api.pyx", line 1215, in openvino.inference_engine.ie_api.IEPlugin.load
RuntimeError: [NOT_FOUND] No plugin is loaded to the Multi-Device. Failed to load the network
tar: stdout.log: file changed as we read it
